In [1]:
import re
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import pandas as pd

In [106]:
# url = 'https://6y.nuc.edu.cn/gyxy/jzg/jzgyl.htm'
url = 'https://6y.nuc.edu.cn/gyxy/jzg/jzgyl/6.htm'
html = requests.get(url, headers={'User-Agent': UserAgent().random}).content
bs0bj = BeautifulSoup(html, 'html.parser')

In [107]:
print(bs0bj)

<html lang="zh-cn">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="仪器与电子" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<link href="" rel="shortcut icon"/>
<title>教职工一览-仪器与电子学院</title>
<!-- Bootstrap core CSS -->
<link href="../../../style/bootstrap.min.css" rel="stylesheet"/>
<link href="../../../style/carousel.jpg.css" rel="stylesheet"/>
<link href="../../../style/all.jpg.css" rel="stylesheet"/>
<!-- Custom styles for this template -->
<!-- Just for debugging purposes. Don't actually copy this line! -->
<!--[if lt IE 9]><script src="../../docs-assets/js/ie8-responsive-file-warning.js"></script><![endif]-->
<!-- HTML5 shim and Respond.js IE8 support of HTML5 elements and media queries -->
<!--[if lt IE 9]>
      <script src="http://cdn.bootcss.com/html5shiv/3.7.0/html5shiv.m

In [123]:
infos = bs0bj.find_all('div', {'class': 'media col-xs-6'})
print(infos)
# details = bs0bj.find_all('div', {'class': 'modal fade'})

[<div class="media col-xs-6" id="line_u6_0" style="display:none;margin-top:0px">
<img alt="" class="media-object pull-left thumbnail" height="180" src="/__local/B/85/1B/E3B62541D1E6FE87BCF018F386B_9E78039A_E52B.png" width="136"/>
<div class="media-body">
<ul class="ul-teacher-detail">
<li>姓名：<a data-target="#myModa13768" data-toggle="modal" href="javascript:void(0)">郭肖亭</a></li>
<li>性别：女</li>
<li>学历：博士</li>
<li>职称职务：讲师</li>
<li>电话：</li>
<li>电子邮件：guoxiaoting@nuc.edu.cn</li></ul></div>
</div>, <div class="media col-xs-6" id="line_u6_1" style="display:none;margin-top:0px">
<img alt="" class="media-object pull-left thumbnail" height="180" src="/__local/E/A8/9E/084B9F2F08A950C1D85D116CA45_1DDBF266_B5BA.jpg" width="136"/>
<div class="media-body">
<ul class="ul-teacher-detail">
<li>姓名：<a data-target="#myModa8986" data-toggle="modal" href="javascript:void(0)">何常德</a></li>
<li>性别：男</li>
<li>学历：博士</li>
<li>职称职务：讲师</li>
<li>电话：0351-3920399</li>
<li>电子邮件：hechangde@nuc.edu.cn</li></ul></div>
</div>

In [76]:
def extract_info(text):
    cut = ['教育经历', '研究方向', '研究成果', '招生方向', '']
    middle = ['：(.*?)', '：(.*?)', '：(.*?)', '：(.*)']
    # print(under)
    detail_dict = {}
    for index in range(4):
        pattern = r'' + cut[index] + middle[index] + cut[index+1]
        content = re.findall(pattern, text)
        detail_dict[cut[index]] = content
    return detail_dict

In [129]:
info_list = [] 
## 全部的循环，每一次都可以拿到126条信息
for info in infos:
    # 从基本中拿到基本信息
    items = info.find_all('li')
    target_id = items[0].find('a')['data-target'].replace('#', '')
    print(target_id)
    teacher_dict = {}
    for item in items:
        key, value = item.get_text().split('：')
        # print(key, value)
        teacher_dict[key] = value

    # 根据序号匹配details
    detail = bs0bj.find('div', {'id': target_id})

    # details 中的学科和信息
    first = detail.find('ul').find_all('li')
    for item in first:
        if '所属学科' in item.get_text():
            teacher_dict['学科'] = item.get_text().replace('所属学科：', '')
        if '所属专业' in item.get_text():
            teacher_dict['专业'] = item.get_text().replace('所属专业：', '')

    # 其他信息（有两种情况）
    under = detail.find('div', id='teacher-dialog-bottom').get_text().replace('\n', '').replace(' ', '').replace(
        '\t', '').replace('\r', '')
    if '教育经历' in under:
        detail_dict = extract_info(under)
        teacher_dict.update(detail_dict)
    else:
        teacher_dict['详细信息'] = under
    info_list.append(teacher_dict)
info_df = pd.DataFrame(info_list)
print(info_df)

myModa13768
myModa8986
myModa13845
myModa17937
myModa9004
myModa9000
myModa8998
myModa8987
myModa22649
myModa13771
myModa9053
myModa13781
myModa22650
myModa13782
     姓名 性别     学历    职称职务            电话                     电子邮件  \
0   郭肖亭  女     博士      讲师                 guoxiaoting@nuc.edu.cn   
1   何常德  男     博士      讲师  0351-3920399     hechangde@nuc.edu.cn   
2    何剑  男     博士   教授、博导  0351-3924575      drhejian@nuc.edu.cn   
3   贺文君  女     博士      讲师  0351-3924575      hewenjun@nuc.edu.cn   
4   洪应平  男     博士     副教授                hongyingping@nuc.edu.cn   
5   侯晓娟  女     博士      教授                 houxiaojuan@nuc.edu.cn   
6   侯钰龙  男     博士   高级实验师  0351-3558768     houyulongnuc@163.com   
7    侯卓  男     博士      讲师                     houzhuo@nuc.edu.cn   
8   韩卫利  女  硕士研究生          0351-3922540                            
9    黄宁  男     本科   综合办主任  0351-3922143      20002131@nuc.edu.cn   
10  焦新泉  男     博士      教授                 jiaoxinquan@nuc.edu.cn   
11   金丽  女     博士     

In [103]:
for info in infos: 
    # 从基本中拿到基本信息
    items = info.find_all('li')
    # print(items[0])
    item = items[0]
    print(item.find('a')['data-target'])

#myModa23390
#myModa9099
#myModa9096
#myModa9094
#myModa17850
#myModa9091
#myModa9077
#myModa9041
#myModa9097
#myModa21979
#myModa9093
#myModa9092
#myModa9055
#myModa9089


In [75]:
# detail = details[0]
# under = detail.find('div', id = 'teacher-dialog-bottom').get_text().replace('\n', '')
# # extract_info(under)
# cut = ['教育经历', '研究方向', '研究成果', '招生方向', '']
# middle = ['：(.*?)', '：(.*?)', '：(.*?)', '：(.*)']
# # print(under)
# detail_dict = {}
# for index in range(4):
#     pattern = r'' + cut[index] + middle[index] + cut[index+1]
#     content = re.findall(pattern, under)
#     detail_dict[cut[index]] = content
#     print(pattern)
# 
# print(detail_dict)

教育经历：(.*?)研究方向
研究方向：(.*?)研究成果
研究成果：(.*?)招生方向
招生方向：(.*)
{'教育经历': ['1995年北京理工大学博士毕业，1998年清华大学出站博士后，1999年美国加州伯克利大学工学院伯克利传感器与执行器研究中心高级访问学者，2001年日本计量国家实验室高级访问学者，2010年麻省理工学院高级访问学者，美国IEEE高级会员\xa0'], '研究方向': ['动态测试技术与智能仪器、微型机电系统（MEMS）等\xa0'], '研究成果': ['先后主持完成973、国家863、国家自然科学基金重点项目、国家自然科学基金重大仪器专项等科研项目40余项。获得国家科学技术发明二等奖3项（排名第一2项，第二1项）、国家科学技术进步二等奖1项。目前，已发表学术论文102篇，其中有57篇被SCI收录，出版专著4部，授权国家发明专利24项。研究成果已在兵器、航空、航天、航海等领域46个国家重大和重点工程得到广泛的应用。“全国百篇优秀博士论文”，“国家杰出青年科学基金”获得者。由于研究成果和工作成绩突出，先后被授予 “国家有突出贡献的中青年专家”称号、“第5届中国青年科技奖”章， “全国五一劳动奖” 章， “全国优秀骨干教师”称号，何梁何利基金“科学技术创新奖”章。\xa0'], '招生方向': ['动态测试技术与智能仪器、微型机电系统（MEMS）等']}


In [54]:
print(under)

教育经历：1995年北京理工大学博士毕业，1998年清华大学出站博士后，1999年美国加州伯克利大学工学院伯克利传感器与执行器研究中心高级访问学者，2001年日本计量国家实验室高级访问学者，2010年麻省理工学院高级访问学者，美国IEEE高级会员 研究方向：动态测试技术与智能仪器、微型机电系统（MEMS）等 研究成果：先后主持完成973、国家863、国家自然科学基金重点项目、国家自然科学基金重大仪器专项等科研项目40余项。获得国家科学技术发明二等奖3项（排名第一2项，第二1项）、国家科学技术进步二等奖1项。目前，已发表学术论文102篇，其中有57篇被SCI收录，出版专著4部，授权国家发明专利24项。研究成果已在兵器、航空、航天、航海等领域46个国家重大和重点工程得到广泛的应用。“全国百篇优秀博士论文”，“国家杰出青年科学基金”获得者。由于研究成果和工作成绩突出，先后被授予 “国家有突出贡献的中青年专家”称号、“第5届中国青年科技奖”章， “全国五一劳动奖” 章， “全国优秀骨干教师”称号，何梁何利基金“科学技术创新奖”章。 招生方向：动态测试技术与智能仪器、微型机电系统（MEMS）等


In [93]:
print(info_list)
len(info_list)

[{'姓名': '张文栋', '性别': '男', '学历': '博士', '职称职务': '教授/国家重点实验室主任', '电话': '0351-3922131', '电子邮件': 'wdzhang@nuc.edu.cn', '学科': '仪器科学与技术、电子科学与技术', '专业': '测控技术与仪器专业、电子科学与技术、微电子学', '教育经历': ['1995年北京理工大学博士毕业，1998年清华大学出站博士后，1999年美国加州伯克利大学工学院伯克利传感器与执行器研究中心高级访问学者，2001年日本计量国家实验室高级访问学者，2010年麻省理工学院高级访问学者，美国IEEE高级会员\xa0'], '研究方向': ['动态测试技术与智能仪器、微型机电系统（MEMS）等\xa0'], '研究成果': ['先后主持完成973、国家863、国家自然科学基金重点项目、国家自然科学基金重大仪器专项等科研项目40余项。获得国家科学技术发明二等奖3项（排名第一2项，第二1项）、国家科学技术进步二等奖1项。目前，已发表学术论文102篇，其中有57篇被SCI收录，出版专著4部，授权国家发明专利24项。研究成果已在兵器、航空、航天、航海等领域46个国家重大和重点工程得到广泛的应用。“全国百篇优秀博士论文”，“国家杰出青年科学基金”获得者。由于研究成果和工作成绩突出，先后被授予“国家有突出贡献的中青年专家”称号、“第5届中国青年科技奖”章，“全国五一劳动奖”章，“全国优秀骨干教师”称号，何梁何利基金“科学技术创新奖”章。\xa0'], '招生方向': ['动态测试技术与智能仪器、微型机电系统（MEMS）等'], '详细信息': '教育经历：1995年北京理工大学博士毕业，1998年清华大学出站博士后，1999年美国加州伯克利大学工学院伯克利传感器与执行器研究中心高级访问学者，2001年日本计量国家实验室高级访问学者，2010年麻省理工学院高级访问学者，美国IEEE高级会员\xa0研究方向：动态测试技术与智能仪器、微型机电系统（MEMS）等\xa0研究成果：先后主持完成973、国家863、国家自然科学基金重点项目、国家自然科学基金重大仪器专项等科研项目40余项。获得国家科学技术发明二等奖3项（排名第一2项，第二1项）、国家科学技术进步二等奖1项。目前，已发表学术论文102篇，其中有

14

In [99]:
def exist_info(info_list, name):
    names = [info['姓名'] for info in info_list]
    return name in names

In [97]:
name = items[0].get_text().replace('姓名：', '')
print(name)
info_list = []

张志杰


In [128]:
bs0bj.find('div', {'id': 'myModa13768'})

<div aria-hidden="true" aria-labelledby="myModalLabel" class="modal fade" id="myModa13768" role="dialog" tabindex="-1">
<div class="modal-dialog">
<div class="modal-content">
<div class="modal-header">
<button aria-hidden="true" class="close" data-dismiss="modal" type="button">×</button>
<h4 class="modal-title" id="myModalLabel">教师信息</h4>
</div>
<div class="modal-body">
<div class="media">
<img alt="" class="media-object pull-left thumbnail" height="220px" src="/__local/B/85/1B/E3B62541D1E6FE87BCF018F386B_9E78039A_E52B.png" width="164px"/>
<div class="media-body">
<ul class="ul-teacher-detail">
<li id="13768">性别：女</li>
<li id="13768">学历：博士</li>
<li id="13768">职称职务：讲师</li>
<li id="13768">办公电话：</li>
<li id="13768">电子邮件：guoxiaoting@nuc.edu.cn</li>
<li id="13768">所属学科：仪器科学与技术</li>
<li id="13768">所属专业：测控技术与仪器</li>
</ul>
</div>
</div>
<div id="teacher-dialog-bottom">
<div id="vsb_content_13768_u51"><div id="vsb_content"><p style="margin-bottom: 15px;line-height: 2em"><strong>教育经历： </strong>2

In [118]:
print(infos)

[<div class="media col-xs-6" id="line_u6_0" style="display:none;margin-top:0px">
<img alt="" class="media-object pull-left thumbnail" height="180" src="/__local/B/85/1B/E3B62541D1E6FE87BCF018F386B_9E78039A_E52B.png" width="136"/>
<div class="media-body">
<ul class="ul-teacher-detail">
<li>姓名：<a data-target="#myModa13768" data-toggle="modal" href="javascript:void(0)">郭肖亭</a></li>
<li>性别：女</li>
<li>学历：博士</li>
<li>职称职务：讲师</li>
<li>电话：</li>
<li>电子邮件：guoxiaoting@nuc.edu.cn</li></ul></div>
</div>, <div class="media col-xs-6" id="line_u6_1" style="display:none;margin-top:0px">
<img alt="" class="media-object pull-left thumbnail" height="180" src="/__local/E/A8/9E/084B9F2F08A950C1D85D116CA45_1DDBF266_B5BA.jpg" width="136"/>
<div class="media-body">
<ul class="ul-teacher-detail">
<li>姓名：<a data-target="#myModa8986" data-toggle="modal" href="javascript:void(0)">何常德</a></li>
<li>性别：男</li>
<li>学历：博士</li>
<li>职称职务：讲师</li>
<li>电话：0351-3920399</li>
<li>电子邮件：hechangde@nuc.edu.cn</li></ul></div>
</div>

In [ ]:
re.findall()